**CREDIT CARD LEAD PREDICTION:**
Happy Customer Bank is a mid-sized private bank that deals in all kinds of banking products, like Savings accounts, Current accounts, investment products, credit products, among other offerings. The bank also cross-sells products to its existing customers and to do so they use different kinds of communication like tele-calling, e-mails, recommendations on net banking, mobile banking, etc. In this case, the Happy Customer Bank wants to cross sell its credit cards to its existing customers. The bank has identified a set of customers that are eligible for taking these credit cards. Now, the bank is looking for your help in identifying customers that could show higher intent towards a recommended credit card, given:

- Customer details (gender, age, region etc.)
- Details of his/her relationship with the bank (Channel_Code,Vintage, 'Avg_Asset_Value etc.)


**IMPORT THE LIBRARIES**

In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/content/train.csv')

In [ ]:
train.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [ ]:
train.shape

(245725, 11)

In [ ]:
train.isnull().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         29325
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245725 entries, 0 to 245724
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ID                   245725 non-null  object
 1   Gender               245725 non-null  object
 2   Age                  245725 non-null  int64 
 3   Region_Code          245725 non-null  object
 4   Occupation           245725 non-null  object
 5   Channel_Code         245725 non-null  object
 6   Vintage              245725 non-null  int64 
 7   Credit_Product       216400 non-null  object
 8   Avg_Account_Balance  245725 non-null  int64 
 9   Is_Active            245725 non-null  object
 10  Is_Lead              245725 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 20.6+ MB


In [ ]:
train['Credit_Product'].unique()

array(['No', nan, 'Yes'], dtype=object)

In [ ]:
train.columns

Index(['ID', 'Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
       'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active',
       'Is_Lead'],
      dtype='object')

In [ ]:
train.drop('ID',axis=1,inplace=True)

**DATA PREPROCESSING**

In [ ]:
categorical_var=['Gender', 'Region_Code', 'Occupation', 'Channel_Code','Credit_Product', 'Is_Active']

In [ ]:
for c in categorical_var :
  train[c] = pd.get_dummies(train[c])


In [ ]:
train

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,1,73,0,0,0,43,1,1045696,1,0
1,1,30,0,0,1,32,1,581988,1,0
2,1,56,0,0,0,26,1,1484315,0,0
3,0,34,0,0,1,19,1,470454,1,0
4,1,30,0,0,1,33,1,886787,1,0
...,...,...,...,...,...,...,...,...,...,...
245720,0,51,0,0,0,109,0,1925586,1,0
245721,0,27,0,0,1,15,1,862952,0,0
245722,1,26,0,0,1,13,1,670659,1,0
245723,1,28,0,0,1,31,1,407504,1,0


In [ ]:
 train['Credit_Product'].fillna((train['Credit_Product'].mean()),inplace=True)

In [ ]:
train['Credit_Product'].values

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [ ]:
train.isnull().sum()

Gender                 0
Age                    0
Region_Code            0
Occupation             0
Channel_Code           0
Vintage                0
Credit_Product         0
Avg_Account_Balance    0
Is_Active              0
Is_Lead                0
dtype: int64

In [ ]:
train['Is_Lead'].value_counts()

0    187437
1     58288
Name: Is_Lead, dtype: int64

In [ ]:
train.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,1,73,0,0,0,43,1,1045696,1,0
1,1,30,0,0,1,32,1,581988,1,0
2,1,56,0,0,0,26,1,1484315,0,0
3,0,34,0,0,1,19,1,470454,1,0
4,1,30,0,0,1,33,1,886787,1,0


In [ ]:
y=train['Is_Lead']

In [ ]:
train.drop('Is_Lead',axis=1,inplace=True)

In [ ]:
train=train.to_numpy()

In [ ]:
y=y.to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=(train_test_split(train,y,test_size=0.30))

In [ ]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
X_train.shape

(172007, 9, 1)

**CONVOLUTION NEURAL NETWORK MODEL**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.optimizers import Adam

In [ ]:
model=Sequential()
model.add(Conv1D(32,2,activation='relu',input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64,2,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 32)             96        
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 32)             128       
_________________________________________________________________
dropout (Dropout)            (None, 8, 32)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7, 64)             4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 64)             256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 448)               0

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[tf.keras.metrics.AUC(),"accuracy"])

In [ ]:
res=model.fit(X_train,Y_train,epochs=50,validation_data=(X_test,Y_test),verbose=1)

Epoch 1/50
5376/5376 [==============================] - 62s 6ms/step - loss: 0.5293 - auc: 0.6373 - accuracy: 0.7614 - val_loss: 0.5073 - val_auc: 0.6737 - val_accuracy: 0.7625
Epoch 2/50
5376/5376 [==============================] - 30s 6ms/step - loss: 0.5178 - auc: 0.6573 - accuracy: 0.7629 - val_loss: 0.5039 - val_auc: 0.6640 - val_accuracy: 0.7625
Epoch 3/50
5376/5376 [==============================] - 30s 6ms/step - loss: 0.5172 - auc: 0.6597 - accuracy: 0.7629 - val_loss: 0.5041 - val_auc: 0.6720 - val_accuracy: 0.7625
Epoch 4/50
5376/5376 [==============================] - 30s 6ms/step - loss: 0.5157 - auc: 0.6609 - accuracy: 0.7630 - val_loss: 0.5038 - val_auc: 0.6749 - val_accuracy: 0.7625
Epoch 5/50
5376/5376 [==============================] - 30s 6ms/step - loss: 0.5150 - auc: 0.6636 - accuracy: 0.7638 - val_loss: 0.4995 - val_auc: 0.6807 - val_accuracy: 0.7625
Epoch 6/50
5376/5376 [==============================] - 30s 6ms/step - loss: 0.5124 - auc: 0.6687 - accuracy: 0.764

**PREDICTION**

In [ ]:
test=pd.read_csv('/content/test.csv')

In [ ]:
test.shape

(105312, 10)

In [ ]:
test.isnull().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         12522
Avg_Account_Balance        0
Is_Active                  0
dtype: int64

In [ ]:
test.drop('ID',axis=1,inplace=True)

In [ ]:
categorical_var=['Gender', 'Region_Code', 'Occupation', 'Channel_Code','Credit_Product', 'Is_Active']

In [ ]:
for c in categorical_var :
  test[c] = pd.get_dummies(test[c])

In [ ]:
 test['Credit_Product'].fillna((test['Credit_Product'].mean()),inplace=True)

In [ ]:
test.isnull().sum()

Gender                 0
Age                    0
Region_Code            0
Occupation             0
Channel_Code           0
Vintage                0
Credit_Product         0
Avg_Account_Balance    0
Is_Active              0
dtype: int64

In [ ]:
test.head()

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,0,29,0,0,1,25,0,742366,1
1,0,43,0,0,0,49,0,925537,1
2,0,31,0,0,1,14,1,215949,1
3,0,29,0,0,1,33,1,868070,1
4,1,29,0,0,1,19,1,657087,1


In [ ]:
test=test.to_numpy()

In [ ]:
test=test.reshape(test.shape[0],test.shape[1],1)


In [ ]:
y_pred=model.predict(test)


In [ ]:
y_pred

array([[0.325758  ],
       [0.5029721 ],
       [0.04785087],
       ...,
       [0.11910138],
       [0.11990245],
       [0.04876754]], dtype=float32)

In [ ]:
DF=pd.DataFrame(y_pred)

In [ ]:
DF.to_csv('prediction_CNN2.csv')